In [3]:
## Example
import numpy as np
import meshplot as mp



In [4]:
from meshLoader import MeshLoader
import torch
from pytorch3d.datasets import ShapeNetCore
from pytorch3d.structures import Meshes
from pytorch3d.renderer import TexturesVertex
import matplotlib.pyplot as plt
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes, load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras, 
    PointLights, 
    DirectionalLights, 
    Materials, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    SoftPhongShader,
    TexturesUV,
    TexturesVertex
)

import numpy as np
import open3d as o3d
from open3d.web_visualizer import draw


if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
        
SHAPENET_PATH = "/home/ghandour/Dataset/ShapeNetCore.v2"
shapenet_dataset = ShapeNetCore(SHAPENET_PATH, version=2)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


/home/ghandour/miniconda3/envs/dd3/lib/python3.9/site-packages/pytorch3d/datasets/shapenet/shapenet_core.py:127: UserWarning: Object file not found in the model directory 207e69af994efa9330714334794526d4 under synset directory 02958343.
  warnings.warn(msg)
/home/ghandour/miniconda3/envs/dd3/lib/python3.9/site-packages/pytorch3d/datasets/shapenet/shapenet_core.py:127: UserWarning: Object file not found in the model directory 5bf2d7c2167755a72a9eb0f146e94477 under synset directory 02958343.
  warnings.warn(msg)
/home/ghandour/miniconda3/envs/dd3/lib/python3.9/site-packages/pytorch3d/datasets/shapenet/shapenet_core.py:127: UserWarning: Object file not found in the model directory 8070747805908ae62a9eb0f146e94477 under synset directory 02958343.
  warnings.warn(msg)
/home/ghandour/miniconda3/envs/dd3/lib/python3.9/site-packages/pytorch3d/datasets/shapenet/shapenet_core.py:127: UserWarning: Object file not found in the model directory e6c22be1a39c9b62fb403c87929e1167 under synset directory

In [5]:
pcd = MeshLoader(shapenet_dataset)

## Create Mesh
shapenet_model = shapenet_dataset[0]

print(shapenet_model.keys())
print("This model belongs to the category " + shapenet_model["synset_id"] + ".")
print("This model has model id " + shapenet_model["model_id"] + ".")
print("This model has a label of: ", shapenet_model["label"])

dict_keys(['synset_id', 'model_id', 'verts', 'faces', 'textures', 'label'])
This model belongs to the category 03691459.
This model has model id 69212e8c41ec753fbe62db7ad46c40f3.
This model has a label of:  loudspeaker


/home/ghandour/miniconda3/envs/dd3/lib/python3.9/site-packages/pytorch3d/io/mtl_io.py:102: UserWarning: Texture UV coordinates outside the range [0, 1].                 The integer part will be ignored to form a repeating pattern.
  warnings.warn(msg)


In [6]:
print("Model with faces and vertices")
mp.plot(pcd[0]["verts"].numpy(), pcd[0]["faces"].numpy())

print("Model with just vertices")
mp.plot(pcd[0]["verts"].numpy())

print("Model with resampled verts")
mp.plot(pcd[0]["coordinates"].numpy())


Model with faces and vertices


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0013030…

Model with just vertices


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0013030…

Model with resampled verts


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.317195…

In [24]:
import open3d as o3d


tri_mesh = o3d.geometry.TriangleMesh()
tri_mesh.vertices = o3d.utility.Vector3dVector(pcd[0]["verts"])
tri_mesh.triangles = o3d.utility.Vector3iVector(pcd[0]["faces"])
voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(tri_mesh, voxel_size=0.005)
voxels = voxel_grid.get_voxels()
print(voxels[0].grid_index)

[110  73  50]


In [33]:
indices = np.stack(list(vx.grid_index for vx in voxels))


shading = {"flat":True, # Flat or smooth shading of triangles
           "wireframe":False, "wire_width": 0.03, "wire_color": "black", # Wireframe rendering
           "width": 600, "height": 600, # Size of the viewer canvas
           "antialias": True, # Antialising, might not work on all GPUs
           "scale": 2.0, # Scaling of the model
           "side": "DoubleSide", # FrontSide, BackSide or DoubleSide rendering of the triangles
           "colormap": "viridis", "normalize": [None, None], # Colormap and normalization for colors
           "background": "#ffffff", # Background color of the canvas
           "line_width": 1.0, "line_color": "black", # Line properties of overlay lines
           "bbox": False, # Enable plotting of bounding box
           "point_color": "red", "point_size": 1 # Point properties of overlay points
          }
mp.plot(indices, shading=shading)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(81.0, 41.…

In [53]:
def resample_mesh(faces=None, vertices=None, mesh_cad=None, density=0.1):
    """
    https://chrischoy.github.io/research/barycentric-coordinate-for-mesh-sampling/
    Samples point cloud on the surface of the model defined as vectices and
    faces. This function uses vectorized operations so fast at the cost of some
    memory.

    param mesh_cad: low-polygon triangle mesh in o3d.geometry.TriangleMesh
    param density: density of the point cloud per unit area
    param return_numpy: return numpy format or open3d pointcloud format
    return resampled point cloud

    Reference :
      [1] Barycentric coordinate system
      \begin{align}
        P = (1 - \sqrt{r_1})A + \sqrt{r_1} (1 - r_2) B + \sqrt{r_1} r_2 C
      \end{align}
    """

    if(mesh_cad is not None):
        faces = np.array(mesh_cad.triangles).astype(int)
        vertices = np.array(mesh_cad.vertices)

    vec_cross = np.cross(
        vertices[faces[:, 0], :] - vertices[faces[:, 2], :],
        vertices[faces[:, 1], :] - vertices[faces[:, 2], :],
    )
    face_areas = np.sqrt(np.sum(vec_cross ** 2, 1))

    n_samples = (np.sum(face_areas) * density).astype(int)
    # face_areas = face_areas / np.sum(face_areas)

    # Sample exactly n_samples. First, oversample points and remove redundant
    # Bug fix by Yangyan (yangyan.lee@gmail.com)
    n_samples_per_face = np.ceil(density * face_areas).astype(int)
    floor_num = np.sum(n_samples_per_face) - n_samples
    if floor_num > 0:
        indices = np.where(n_samples_per_face > 0)[0]
        floor_indices = np.random.choice(indices, floor_num, replace=True)
        n_samples_per_face[floor_indices] -= 1

    n_samples = np.sum(n_samples_per_face)

    # Create a vector that contains the face indices
    sample_face_idx = np.zeros((n_samples,), dtype=int)
    acc = 0
    for face_idx, _n_sample in enumerate(n_samples_per_face):
        sample_face_idx[acc : acc + _n_sample] = face_idx
        acc += _n_sample

    r = np.random.rand(n_samples, 2)
    A = vertices[faces[sample_face_idx, 0], :]
    B = vertices[faces[sample_face_idx, 1], :]
    C = vertices[faces[sample_face_idx, 2], :]

    P = (
        (1 - np.sqrt(r[:, 0:1])) * A
        + np.sqrt(r[:, 0:1]) * (1 - r[:, 1:]) * B
        + np.sqrt(r[:, 0:1]) * r[:, 1:] * C
    )

    return P

In [58]:
verts, faces = pcd[0]["verts"], pcd[0]["faces"]

sampled_verts = resample_mesh(mesh_cad=tri_mesh, density=10000)
mp.plot(sampled_verts)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0013016…